# Merging LandCover.ai and DeepGlobe dataset

### todo
- [ ] find fitting resolution
- [ ] transfer labels to same
- [ ] Resize resolution
- [ ] v0 verwenden weil da keine straßen als label sind??

### additional info

using landcover.ai v1

In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
# data root
LANDCOVER_AI_ROOT = '../data/landcover.ai.v1'
DEEPGLOBE_ROOT = '../data/deepglobe'
MERGED_ROOT = '../data/merge'

Landcover.ai dataset masks need to be mapped from class to color to visualise the pictures 

In [ ]:
from notebooks.utils import split_images
# cropping images due to gpu resource and the need of fixed size for the algorithm
# split_images(LANDCOVER_AI_ROOT)
# split_images(DEEPGLOBE_ROOT + '/train')
# split_images(FIRST_LANDCOVER_AI_ROOT + '/train')

In [ ]:
import notebooks.utils as ut

metadata_df = ut.read_metadata('../data/merge')
file_name = '../data/metadata.xlsx'
metadata_df.to_excel(file_name, index=False, na_rep='N/A', index_label='ID')

In [ ]:
# Perform 90/10 split for train / val
valid_df = metadata_df.sample(frac=0.2, random_state=42)
train_df = metadata_df.drop(valid_df.index)
len(train_df), len(valid_df)

In [ ]:
# FIRST_LANDCOVER_AI_ROOT = '../data/landcover.ai.v0'
# split_images(FIRST_LANDCOVER_AI_ROOT)


In [14]:
# pytorsch
# batch normalisier
# normalisieren wertebereich
# euqalizer nur schwarz weiß
# noise reducting min max 
# blurr
# one hot
# yolo vs cnn
#

urban_land,0,255,255
agriculture_land,255,255,0
rangeland,255,0,255
forest_land,0,255,0
water,0,0,255
barren_land,255,255,255
unknown,0,0,0

building (1), woodland (2), water(3), road(4)
->
building 1 -> 0,255,255
woodland 2 -> 0,255,0
water    3 -> 0,0,255
road     4 -> 0,0,0
